<a href="https://colab.research.google.com/github/stepiglia/data_science_python/blob/main/Google_Trends_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Source article](https://searchengineland.com/learn-how-to-chart-and-track-google-trends-in-data-studio-using-python-329119) 

##Install PyTrends

In [ ]:
pip install pytrends

In [ ]:
import pytrends
from pytrends.request import TrendReq
import pandas as pd
import time
import datetime
from datetime import datetime, date, time

pytrend = TrendReq()

##Find suggested Keywords to add to the analysis

In [ ]:
#return  suggested keywords and their ‘’types.’’
print(pytrend.suggestions(keyword='migliori antifurto'), '\n')
print(pytrend.suggestions(keyword='assicurazione viaggio'), '\n')

[] 

[{'mid': '/m/08jvtd', 'title': 'Travel insurance', 'type': 'Insurance category'}, {'mid': '/g/11bc71341d', 'title': "Holins - l'Assicurazione viaggio 100% Italiano", 'type': 'Insurance agency in Turin, Italy'}] 



##Return related keywords

In [ ]:
#Can be really helpful in Google Ads, keyword research and content creation.
#Remember that you can add a category after the timeframe as   , cat = 44
pytrend.build_payload(kw_list=['antifurto'], geo = 'IT', timeframe = 'today 12-m') 
related_queries= pytrend.related_queries()
print(related_queries)

{'antifurto': {'top':                          query  value
0               antifurto casa    100
1               antifurto auto     93
2                sim antifurto     42
3               antifurto moto     30
4            sim per antifurto     27
5               antifurto bici     26
6            bulloni antifurto     24
7                 allarme casa     21
8                antifurto 365     20
9         antifurto senza fili     19
10          impianto antifurto     16
11          antifurto per auto     16
12          antifurto wireless     14
13            antifurto bentel     14
14       antifurto perimetrale     13
15             zaino antifurto     13
16              ajax antifurto     12
17   antifurto casa senza fili     11
18          impianti antifurto     11
19  antifurto satellitare auto     10
20           squadra antifurto     10
21       sim per antifurto tim      9
22          antifurto verisure      9
23           bullock antifurto      9
24         antifurto meccani

##Final analysis with many Keywords (WIP, seems off as expected)

In [ ]:
#Remember that you can also read keywords from a csv or excel file but make sure that its type must be a ‘’list.’’
searches = ['prestito 10000 euro','prestiti online','migliori antifurto','migliori chirurghi plastici','migliori avvocati','assicurazione viaggio','migliori banche mutuo','antifurto casa','antifurto monopattino']

groupkeywords = list(zip(*[iter(searches)]*1))
groupkeywords = [list(x) for x in groupkeywords]

dicti = {}
i = 1
for trending in groupkeywords:
  pytrend.build_payload(trending, timeframe = 'today 12-m', geo = 'IT')
  dicti[i] = pytrend.interest_over_time()
  i+=1

result = pd.concat(dicti, axis=1)
result.columns = result.columns.droplevel(0)
result = result.drop('isPartial', axis = 1)




In [ ]:
#Reshape in tabular form 
result.reset_index(level=0, inplace=True)
result = pd.melt(result, id_vars='date', value_vars=searches)

## Compare 10 keywords

In [ ]:
#Import TrendReq to connect to Google
from pytrends.request import TrendReq

list1=['The office','Stranger Things','Friends', 'Riverdale','Game of Thrones']
list2=['The office', 'Breaking Bad' ,'Money Heist','Peaky Blinders ,'Black Mirror']

#Connect to Google 
pytrends1=TrendReq()
pytrends2=TrendReq()

#Parameters time and geo
pytrends1.build_payload(list1,geo='US',timeframe="today 12-m")
pytrends2.build_payload(list2,geo='US',timeframe="today 12-m")

#using method interest_over_time to get the historical data
df1=pytrends1.interest_over_time()
df2=pytrends2.interest_over_time()

In [1]:
#Normalizing List 2 using list 1 as reference 
averageList1=[]
averageList2=[]
for item in list1:
    averageList1.append(df1[item].mean().round(0))
for item in list2:
    averageList2.append(df2[item].mean().round(0))

NameError: ignored

In [ ]:
#Calculate the normalization factor (i.e. Normalization factor= (average of common TV Series in list 1)/average of common TV Series in list 2)
normalizationFactor=averageList1[0]/averageList2[0] #the common value is at index 0 in both lists 

In [ ]:
#We equalize the averages in list 2 
for i in range(len(averageList2)):
    normalisedVal=normalizationFactor*averageList2[i]
    averageList2[i]=normalisedVal.round(0)

#The two lists are now scaled at the same scale     

In [ ]:
#Remove the now duplicate common list 
averageList2.pop(0)
list2.pop(0)

In [ ]:
#We combine the useful results 
TVSeriesList=list1+list2 #List of TV series 
finalAverageList=averageList1+averageList2 #List of average values 

In [ ]:
#We plot 
import numpy as np
import matplotlib.pyplot as plt
y_pos=np.arange(len(TVSeriesList))
plt.barh(y_pos,finalAverageList,align='center',alpha=0.5)
plt.yticks(y_pos,TVSeriesList)
plt.xlabel('Average popularity')
plt.show()

##Export to Google Sheet

In [ ]:
#!pip install --upgrade gspread

In [ ]:
"""from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
#Documentation for gspread: https://gspread.readthedocs.io/en/latest/user-guide.html#opening-a-spreadsheet
"""


In [ ]:
#result.head()
result["date"] = result["date"].astype(str)
result.head()

In [ ]:
#If starting from scratch: """ sh = gc.create('A new spreadsheet')
sh = gc.open('Trends analysis for data studio')
worksheet = sh.worksheet("RAW")

worksheet.update([result.columns.values.tolist()] + result.values.tolist())